### Create TF graph

In [13]:
# reset
tf.reset_default_graph()

# Parameters
learning_rate = 0.01
training_epochs = 50000
batch_size = 100
display_step = 1000
display_step_state = False

# tf Graph Input
x = tf.placeholder(tf.float32, [None, n_features])
y = tf.placeholder(tf.float32, [None, n_classes])

# Set model weights
W = tf.Variable(tf.zeros([n_features, n_classes]))
b = tf.Variable(tf.zeros([n_classes]))

# Construct model
pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))

# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
# optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

# Test model
prediction = tf.argmax(pred, 1)
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

### Train model

In [14]:
# https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/2_BasicModels/logistic_regression.py

# Start training
with tf.Session() as sess:
    # run the initializer
    sess.run(init)
    X_model_bn, X_test_bn = my.batch_normalization(X_model, X_test, epsilon=.0001)
    # Training cycle starts
    avg_cost = 0
    for epoch in range(training_epochs):  
        # display logs per epoch step
        if ((epoch+1) % display_step == 0) and display_step_state:
            print("Ep:", '%04d' % (epoch+1), 
                  "\n\tcost = {:.5f}\t{:.5f}\t{:.5f}".format(avg_cost/epoch,
                                                             sess.run(accuracy, feed_dict={x: X_model_bn, y: Y_model}),
                                                             sess.run(accuracy, feed_dict={x: X_test_bn, y: Y_test})))
        # 
        _, c = sess.run([optimizer, cost], feed_dict={x: X_model_bn, y: Y_model})

        # aggregate loss
        avg_cost += c

    # model performance
    acc_model = sess.run(accuracy, feed_dict={x: X_model_bn, y: Y_model})
    acc_test = sess.run(accuracy, feed_dict={x: X_test_bn, y: Y_test})

    # save model for folds
    save_path = saver.save(sess, "./../data/models/model_final.ckpt")
    print("Model final saved in path: %s" % save_path)

print("\n\nFinal model performance: \n\tAccuracy model:\t{:.5f}\n\tAccuracy test:\t{:.5f}".format(acc_model, acc_test))

Model final saved in path: ./../data/models/model_final.ckpt


Final model performance: 
	Accuracy model:	0.96316
	Accuracy test:	0.96341


## Model assessment

In [15]:
# get the test accuracy
with tf.Session() as sess:
    # restore variables
    saver.restore(sess, "./../data/models/model_final.ckpt")
    print("Model restored.")
    # batch norm
    _, X_test_bn = my.batch_normalization(X_model, X_test, epsilon=.0001)
    
    # Check the values of the variables
    acc_test, pred_test = sess.run([accuracy, prediction], feed_dict={x: X_test_bn, y: Y_test})
    my.performance(pred_test, Y_test)
    
    Weights, bias = sess.run([W, b])
    

INFO:tensorflow:Restoring parameters from ./../data/models/model_final.ckpt
Model restored.


Accuracy:	0.9634146341463414

Confusion matrix (pred x true)
 [[46  1]
 [ 2 33]]
